In [66]:
from llama_cpp import Llama
from llm_rpg.configs import settings

In [7]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model

In [67]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [68]:
template = """
Jesteś asystentem który odpowiada na pytania użytkownika. Staraj się opdpowiadać na pytanie dokłądnie i zwięźle oraz nie halucynować.
Pytanie: {question}
"""

prompt = PromptTemplate.from_template(template)

In [69]:
n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=f"{settings.models_path}/Bielik-4.5B-v3.0-Instruct.Q8_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=False,  # Verbose is required to pass to the callback manager
)

llama_context: n_ctx_per_seq (512) < n_ctx_train (8192) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h96   

In [57]:
llm_chain = prompt | llm
question = "Kto to robert lewandowski??"
llm_chain.invoke({"question": question})

Robert Lewandowski to polski piłkarz, który gra na pozycji napastnika w klubie Bayern Monachium oraz w reprezentacji Polski.

Urodził się 21 sierpnia 1988 roku w Warszawie.

Lewandowski jest znany ze swojej skuteczności w ataku, wielokrotnie zdobywając tytuły króla strzelców w lidze niemieckiej i polskiej.

W reprezentacji Polski Lewandowski rozegrał ponad 100 meczów, strzelając wiele ważnych bramek, przyczyniając się do sukcesów drużyny narodowej.

'Robert Lewandowski to polski piłkarz, który gra na pozycji napastnika w klubie Bayern Monachium oraz w reprezentacji Polski.\n\nUrodził się 21 sierpnia 1988 roku w Warszawie.\n\nLewandowski jest znany ze swojej skuteczności w ataku, wielokrotnie zdobywając tytuły króla strzelców w lidze niemieckiej i polskiej.\n\nW reprezentacji Polski Lewandowski rozegrał ponad 100 meczów, strzelając wiele ważnych bramek, przyczyniając się do sukcesów drużyny narodowej.'